# ETL: автоматизация подготовки данных семинары

## Урок 2. Введение в подготовку данных для аналитиков. Таблицы фактов и таблицы измерений

1. Скачайте датасет fifа_s2.сsv. Проанализируйте его и определите, какие данные являются неполными. Удалите
ненужные колонки и недостающие значения.

2. Найдите в датафрейме полные дубликаты и удалите их. Значения могут быть одинаковыми, но написаны по-разному. Например, может отличаться размер регистра (заглавные и строчные буквы). Особое внимание уделить колонке с названиями команд.

3. Напишите функцию, которая добавит колонку с разбиением возраста по группам: до 20, от 20 до 30, от 30 до
36 и старше 36. Посчитайте количество футболистов в каждой категории.

In [36]:
import init_spark_env

In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, udf, lower
from pyspark.sql.types import StringType

In [38]:
# Инициализация Spark Session
spark = SparkSession.builder.appName("FIFA Data Preparation").getOrCreate()

In [39]:
spark

In [40]:
# Загрузка данных
df = spark.read.csv("fifa_s2.csv", header=True, inferSchema=True)
df

DataFrame[ID: int, Name: string, Age: int, Nationality: string, Overall: int, Potential: int, Club: string, Value: int, Wage: int, Preferred Foot: string, International Reputation: int, Skill Moves: int, Position: string, Joined: int, Contract Valid Until: string, Height: double, Weight: double, Release Clause: double]

In [41]:
df.show()

+-----+-----------------+---+-------------------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|   ID|             Name|Age|        Nationality|Overall|Potential|               Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|
+-----+-----------------+---+-------------------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
| 1179|        G. Buffon| 40|              Italy|     88|       88|Paris Saint-Germain| 4000|  77|         Right|                       4|          1|      GK|  2018|          01/01/2019|6.333333333| 203.0|        7400.0|
| 5479|         Casillas| 37|              Spain|     82|       82|           FC Porto| 1500|  10|          Left

In [42]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[ID: int, Name: string, Age: int, Nationality: string, Overall: int, Potential: int, Club: string, Value: int, Wage: int, Preferred Foot: string, International Reputation: int, Skill Moves: int, Position: string, Joined: int, Contract Valid Until: string, Height: double, Weight: double, Release Clause: double]>

In [43]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Value: integer (nullable = true)
 |-- Wage: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Position: string (nullable = true)
 |-- Joined: integer (nullable = true)
 |-- Contract Valid Until: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Release Clause: double (nullable = true)



In [44]:
df.count()

2399

In [45]:
# Подсчет отсутствующих значений в каждой колонке
df_missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

# Выводим количество отсутствующих значений
df_missing_values.show()

+---+----+---+-----------+-------+---------+----+-----+----+--------------+------------------------+-----------+--------+------+--------------------+------+------+--------------+
| ID|Name|Age|Nationality|Overall|Potential|Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|Height|Weight|Release Clause|
+---+----+---+-----------+-------+---------+----+-----+----+--------------+------------------------+-----------+--------+------+--------------------+------+------+--------------+
|  0|   0|  0|          7|      0|        0| 241|  257|   0|             0|                      10|          6|      21|     0|                 273|     0|     0|             3|
+---+----+---+-----------+-------+---------+----+-----+----+--------------+------------------------+-----------+--------+------+--------------------+------+------+--------------+



In [46]:
# Удаление колонок с большим количеством отсутствующих значений
columns_to_drop = ['Club', 'Contract Valid Until', 'Value']
df_cleaned = df.drop(*columns_to_drop)

# Удаление строк с отсутствующими значениями
df_cleaned = df_cleaned.dropna()

In [47]:
# Приведение всех строковых колонок к нижнему регистру
string_columns = [field.name for field in df_cleaned.schema.fields if isinstance(field.dataType, StringType)]
for col_name in string_columns:
    df_cleaned = df_cleaned.withColumn(col_name, lower(col(col_name)))

# Удаление полных дубликатов строк
df_no_duplicates = df_cleaned.dropDuplicates()

In [48]:
# Функция для определения возрастной группы
def age_group(age):
    if age < 20:
        return 'Under 20'
    elif 20 <= age <= 30:
        return '20 to 30'
    elif 31 <= age <= 36:
        return '31 to 36'
    else:
        return 'Over 36'

# Регистрация UDF
age_group_udf = udf(age_group, StringType())

# Добавление новой колонки 'AgeGroup'
df_with_age_groups = df_no_duplicates.withColumn('AgeGroup', age_group_udf(col('Age')))

# Подсчет количества футболистов в каждой возрастной группе
age_group_counts = df_with_age_groups.groupBy('AgeGroup').count()

# Вывод результатов
age_group_counts.show()

+--------+-----+
|AgeGroup|count|
+--------+-----+
|31 to 36|  327|
|Under 20|  269|
| Over 36|   14|
|20 to 30| 1744|
+--------+-----+



In [49]:
spark.stop()